In [279]:
import pandas as pd
import numpy as np
import re

In [280]:
# Load the data
routes = pd.read_csv('data/Flights.csv')
ticket_price = pd.read_csv('data/Tickets.csv')
airportsInfo = pd.read_csv('data/Airport_Codes.csv')

/opt/anaconda3/envs/tongConsultinInc/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3508: DtypeWarning: Columns (3,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Flights
- FL_DATE: string to stanarded datetime object
- ORIGIN_CITY_NAME: split into city and state
- DEST_CITY_NAME: split into city and state
- AIR_TIME: 
    - Two; NAN; negative number; number in str
- DISTANCE: to float

In [281]:
routes['FL_DATE'] = pd.to_datetime(routes['FL_DATE'])

# 
routes['ORIGIN_STATE_NAME'] = routes['ORIGIN_CITY_NAME'].str.split(', ').str[1]
routes['ORIGIN_CITY_NAME'] = routes['ORIGIN_CITY_NAME'].str.split(', ').str[0]
routes['DEST_STATE_NAME'] = routes['DEST_CITY_NAME'].str.split(', ').str[1]
routes['DEST_CITY_NAME'] = routes['DEST_CITY_NAME'].str.split(', ').str[0]

# air time column adjustments
routes['AIR_TIME'] = routes['AIR_TIME'].apply(lambda x: 2.0 if x == 'Two' else x)
routes['AIR_TIME'] = routes['AIR_TIME'].apply(lambda x: np.nan if x == 'NAN' or x == '$$$' else x)
routes['AIR_TIME'] = routes['AIR_TIME'].apply(lambda x: 121.0 if x == '121.0' else x)
routes['AIR_TIME'] = routes['AIR_TIME'].astype(float)

# clean and convert distance to float
def distance_to_float(val):
    try:
        float_val = float(val)
        if float_val < 0:
            return -1 * float_val
        return float_val
    except:
        return np.nan
    
routes['DISTANCE'] = routes['DISTANCE'].apply(distance_to_float)

In [282]:
routes

,FL_DATE,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,DESTINATION,DEST_CITY_NAME,DEP_DELAY,ARR_DELAY,CANCELLED,AIR_TIME,DISTANCE,OCCUPANCY_RATE,ORIGIN_STATE_NAME,DEST_STATE_NAME
0,2019-03-02,WN,N955WN,4591,14635,RSW,Fort Myers,11042,CLE,Cleveland,-8.0,-6.0,0.0,143.0,1025.0,0.970000,FL,OH
1,2019-03-02,WN,N8686A,3231,14635,RSW,Fort Myers,11066,CMH,Columbus,1.0,5.0,0.0,135.0,930.0,0.550000,FL,OH
2,2019-03-02,WN,N201LV,3383,14635,RSW,Fort Myers,11066,CMH,Columbus,0.0,4.0,0.0,132.0,930.0,0.910000,FL,OH
3,2019-03-02,WN,N413WN,5498,14635,RSW,Fort Myers,11066,CMH,Columbus,11.0,14.0,0.0,136.0,930.0,0.670000,FL,OH
4,2019-03-02,WN,N7832A,6933,14635,RSW,Fort Myers,11259,DAL,Dallas,0.0,-17.0,0.0,151.0,1005.0,0.620000,FL,TX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1915881,2019-03-23,AA,N903NN,1433,15370,TUL,Tulsa,11057,CLT,Charlotte,-9.0,-6.0,0.0,112.0,NaN,0.794884,OK,NC
1915882,2019-03-24,AA,N965AN,1433,15370,TUL,Tulsa,11057,CLT,Charlotte,-2.0,-1.0,0.0,106.0,NaN,0.538399,OK,NC
1915883,2019-03-25,AA,N979NN,1433,15370,TUL,Tulsa,11057,CLT,Charlotte,-8.0,-25.0,0.0,106.0,NaN,0.955579,OK,NC
1915884,2019-03-26,AA,N872NN,1433,15370,TUL,Tulsa,11057,CLT,Charlotte,-9.0,-6.0,0.0,112.0,NaN,0.595344,OK,NC


## Tickets
- YEAR to int
- clean itin_fare

In [283]:
# year column to int year
ticket_price['YEAR'] = ticket_price['YEAR'].astype(int)

# clean and convert price to float
def find_number(text):
    if type(text) != str:
        return np.nan
    re_result = re.search(r'[\d\.]+', text)
    if re_result is not None:
        return float(re_result.group(0))
    return np.nan

ticket_price['ITIN_FARE'] = ticket_price['ITIN_FARE'].apply(find_number)



In [284]:
ticket_price

,ITIN_ID,YEAR,QUARTER,ORIGIN,ORIGIN_COUNTRY,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,ROUNDTRIP,REPORTING_CARRIER,PASSENGERS,ITIN_FARE,DESTINATION
0,201912723049,2019,1,ABI,US,TX,Texas,1.0,MQ,1.0,736.0,DAB
1,201912723085,2019,1,ABI,US,TX,Texas,1.0,MQ,1.0,570.0,COS
2,201912723491,2019,1,ABI,US,TX,Texas,1.0,MQ,1.0,564.0,MCO
3,201912723428,2019,1,ABI,US,TX,Texas,1.0,MQ,1.0,345.0,LGA
4,201912723509,2019,1,ABI,US,TX,Texas,0.0,MQ,1.0,309.0,MGM
...,...,...,...,...,...,...,...,...,...,...,...,...
1167280,201911284909,2019,1,YAK,US,AK,Alaska,0.0,AS,1.0,244.0,ANC
1167281,201911284959,2019,1,YAK,US,AK,Alaska,1.0,AS,1.0,371.0,JNU
1167282,201911284940,2019,1,YAK,US,AK,Alaska,0.0,AS,1.0,271.0,JNU
1167283,201911284914,2019,1,YAK,US,AK,Alaska,0.0,AS,1.0,603.0,ANC


## AirportsInfo
- COORDINATES into atomic data set
    - first one is longitude
    - second one is latitude

In [285]:
# clean coordinates

airportsInfo['COORDINATES_LONGITUDE'] = airportsInfo['COORDINATES'].apply(lambda x: x.split(', ')[0]).astype(float)
airportsInfo['COORDINATES_LATITUDE'] = airportsInfo['COORDINATES'].apply(lambda x: x.split(', ')[1]).astype(float)
airportsInfo.drop(columns=['COORDINATES'], inplace=True)

In [286]:
airportsInfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55369 entries, 0 to 55368
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   TYPE                   55369 non-null  object 
 1   NAME                   55369 non-null  object 
 2   ELEVATION_FT           48354 non-null  float64
 3   CONTINENT              27526 non-null  object 
 4   ISO_COUNTRY            55122 non-null  object 
 5   MUNICIPALITY           49663 non-null  object 
 6   IATA_CODE              9182 non-null   object 
 7   COORDINATES_LONGITUDE  55369 non-null  float64
 8   COORDINATES_LATITUDE   55369 non-null  float64
dtypes: float64(3), object(6)
memory usage: 3.8+ MB


In [287]:
airportsInfo

,TYPE,NAME,ELEVATION_FT,CONTINENT,ISO_COUNTRY,MUNICIPALITY,IATA_CODE,COORDINATES_LONGITUDE,COORDINATES_LATITUDE
0,heliport,Total Rf Heliport,11.0,NaN,US,Bensalem,NaN,-74.933601,40.070801
1,small_airport,Aero B Ranch Airport,3435.0,NaN,US,Leoti,NaN,-101.473911,38.704022
2,small_airport,Lowell Field,450.0,NaN,US,Anchor Point,NaN,-151.695999,59.949200
3,small_airport,Epps Airpark,820.0,NaN,US,Harvest,NaN,-86.770302,34.864799
4,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,Newport,NaN,-91.254898,35.608700
...,...,...,...,...,...,...,...,...,...
55364,medium_airport,Yingkou Lanqi Airport,0.0,AS,CN,Yingkou,YKH,122.358600,40.542524
55365,medium_airport,Shenyang Dongta Airport,NaN,AS,CN,Shenyang,NaN,123.496002,41.784401
55366,heliport,Sealand Helipad,40.0,EU,GB,Sealand,NaN,1.482500,51.894444
55367,small_airport,Glorioso Islands Airstrip,11.0,AF,TF,Grande Glorieuse,NaN,47.296389,-11.584278
